__Plockar ut VNUM123 i varje lopp och scorar dessa. __

In [1]:
# Common imports
import numpy as np
import os
import pandas as pd

In [2]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [3]:
# DataPiplene är en pythonfil som innehåller den egenkostruerade klassen DataFrameSelector
from DataPipeline import DataFrameSelector
#import joblib

from sklearn.externals import joblib

/Users/ulf/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


__Läser in filen som ska köras__

In [4]:
df1 = pd.read_excel('TestDataV75TillUffeF20200307.xlsx')

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 36 columns):
Datum          82 non-null datetime64[ns]
Arstid         82 non-null int64
Distans        82 non-null int64
Startsatt      82 non-null int64
Lopp           82 non-null int64
Plac           82 non-null int64
Hast           82 non-null int64
V75PROC        82 non-null float64
V_ODDS         82 non-null float64
GRUPP          82 non-null int64
VLP            82 non-null float64
VNUM           82 non-null int64
SVLP           82 non-null float64
VSVLP          82 non-null float64
VPN_SUM        82 non-null float64
VPN_SUM_ORD    82 non-null int64
VPK_SUM        82 non-null float64
VPK_SUM_ORD    82 non-null int64
VLPB           82 non-null float64
SVLPB          82 non-null float64
VSVLPB         82 non-null float64
E_P            82 non-null float64
E_P_Num        82 non-null int64
E_N            82 non-null float64
E_R            82 non-null float64
E_U            82 non-null float64
G_

In [6]:
# Imputerar mode för median

miss_list = ['G_R','A_R','T_R','SP_R','ToR','P_R','TK_R','R_R','Ts_R']

for var in miss_list:
    df1[var] = df1[var].fillna(df1[var].median())


In [7]:
feat_list = ['G_R','A_R','T_R','SP_R','ToR','P_R','TK_R','R_R','Ts_R']

for var in feat_list:
    df1[var] = df1[var].astype('str')

In [8]:
df1 = df1[df1.VNUM.isin([1,2,3])]  

In [9]:
# Importera model som ska användas
rf = joblib.load('Travmodel_vnum123.pkl')

In [10]:
type(rf)

sklearn.linear_model.logistic.LogisticRegression

In [11]:
# Import av datapipeline som ska användas
full_pipeline = joblib.load('Pipeline_vnum123.pkl')

AttributeError: module '__main__' has no attribute 'SetDummyVar'

__Kör modellen och plockar listan med högsta score__

In [35]:
features = full_pipeline.fit_transform(df1)

In [36]:
predict = rf.predict_proba(features)

ValueError: X has 22 features per sample; expecting 36

In [11]:
# Gör om scorad array till en lista - multidimensionell
stack = predict.tolist()

In [12]:
# Sedan plockar vi den scorade sannolikheten att vara vinnare - 1
# Transformerar denna sannolikhet till en dataframe
last = []
for x in stack:
    last.append(x[1])
scored = {'Score':last}
# Konverterar till Dataframe
df_scored = pd.DataFrame.from_dict(scored)

In [13]:
df_predict2 = df1.merge(df_scored,  right_index = True, left_index = True) 

In [14]:
df1_out = df_predict2[['Lopp','Hast','Score']].sort_values(['Score'], ascending = [False])
df1_out

,Lopp,Hast,Score
4,2,8,0.390974
7,3,2,0.355432
1,1,5,0.325466
8,3,5,0.294173
13,5,4,0.286228
19,7,9,0.271612
2,1,8,0.268476
9,4,1,0.242839
18,7,5,0.223406
10,4,6,0.216921


__Export till excel__

In [15]:
df1_out.to_excel('TestScoredTop21_0307.xlsx', index = False)